<a href="https://colab.research.google.com/github/jamesodukoya/BookRecommendationSystem/blob/main/Book_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Book Recommendations

##Specifying a User's Favorite Books

In [13]:
liked_books = ["4408", "31147619", "29983711", "9401317", "9317691", "8153988", "20494944"]

##Finding All Other Users Who Liked the Same Books As a User

In [14]:
!head /content/drive/MyDrive/ColabNotebooks/book_id_map.csv

book_id_csv,book_id
0,34684622
1,34536488
2,34017076
3,71730
4,30422361
5,33503613
6,33517540
7,34467031
8,6383669


In [15]:
csv_book_mapping = {}

with open("/content/drive/MyDrive/ColabNotebooks/book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [16]:
!wc -l /content/drive/MyDrive/ColabNotebooks/goodreads_interactions.csv

228648343 /content/drive/MyDrive/ColabNotebooks/goodreads_interactions.csv


In [17]:
!ls -lh | grep /content/drive/MyDrive/ColabNotebooks/goodreads_interactions

In [18]:
overlap_users = set()

with open("/content/drive/MyDrive/ColabNotebooks/goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")

        if user_id in overlap_users:
            continue

        try:
            rating = int(rating)
        except ValueError:
            continue

        book_id = csv_book_mapping[csv_id]

        if book_id in liked_books and rating >= 4:
                overlap_users.add(user_id)

##Finding All the Books Those Users Read

In [19]:
rec_lines = []

with open("/content/drive/MyDrive/ColabNotebooks/goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")

        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

##Ranking Recommendations

In [20]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [21]:
recs

,user_id,book_id,rating
0,284,977284,3
1,284,890054,4
2,284,837153,3
3,284,1586480,4
4,284,41814,5
...,...,...,...
1530252,873216,127455,0
1530253,873216,10365343,0
1530254,873216,16131077,0
1530255,873216,18781576,0


In [22]:
top_recs = recs["book_id"].value_counts().head(10)
top_recs = top_recs.index.values

books_titles = pd.read_json("/content/drive/MyDrive/ColabNotebooks/books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [23]:
books_titles.head()

,book_id,title,ratings,url,cover_image,mod_title
0,1333909,Good Harbor,10,https://www.goodreads.com/book/show/1333909.Go...,https://s.gr-assets.com/assets/nophoto/book/11...,good harbor
1,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
2,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
3,287140,Runic Astrology: Starcraft and Timekeeping in ...,15,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,runic astrology starcraft and timekeeping in t...
4,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls


##Creating Initial Book Recommendations

In [24]:
books_titles[books_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover_image,mod_title
71398,77203,The Kite Runner,1848782,https://www.goodreads.com/book/show/77203.The_...,https://images.gr-assets.com/books/1484565687m...,the kite runner
386663,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
546297,5107,The Catcher in the Rye,2086945,https://www.goodreads.com/book/show/5107.The_C...,https://images.gr-assets.com/books/1398034300m...,the catcher in the rye
630937,4671,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
838525,5470,1984,2023937,https://www.goodreads.com/book/show/5470.1984,https://images.gr-assets.com/books/1348990566m...,1984
1031472,38447,The Handmaid's Tale,648783,https://www.goodreads.com/book/show/38447.The_...,https://images.gr-assets.com/books/1498057733m...,the handmaids tale
1077226,2657,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
1195873,18143977,All the Light We Cannot See,498685,https://www.goodreads.com/book/show/18143977-a...,https://images.gr-assets.com/books/1451445646m...,all the light we cannot see
1196415,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
1446134,29983711,Pachinko,8161,https://www.goodreads.com/book/show/29983711-p...,https://images.gr-assets.com/books/1462393298m...,pachinko


##Creating Less Generic and More User-Tailored Recommendations

In [28]:
all_recs = recs["book_id"].value_counts()

all_recs = all_recs.to_frame().reset_index()

all_recs.columns = ["book_id", "book_count"]
all_recs.head(5)

all_recs

,book_id,book_count
0,2767052,1092
1,29983711,1089
2,2657,1074
3,3,1048
4,4671,1028
...,...,...
364164,21843400,1
364165,18595019,1
364166,22514204,1
364167,22733082,1


In [30]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
1,29983711,1089,Pachinko,8161,https://www.goodreads.com/book/show/29983711-p...,https://images.gr-assets.com/books/1462393298m...,pachinko,145.315648
7261,35069544,29,This Will Be My Undoing: Living at the Interse...,8,https://www.goodreads.com/book/show/35069544-t...,https://images.gr-assets.com/books/1502099764m...,this will be my undoing living at the intersec...,105.125000
14752,29749094,15,"Superman (DC Icons, #4)",7,https://www.goodreads.com/book/show/29749094-s...,https://s.gr-assets.com/assets/nophoto/book/11...,superman dc icons 4,32.142857
238,4408,327,East of Eden,3447,https://www.goodreads.com/book/show/4408.East_...,https://images.gr-assets.com/books/1323882457m...,east of eden,31.020888
724,9317691,175,The Name of the Wind (The Kingkiller Chronicle...,1043,https://www.goodreads.com/book/show/9317691-th...,https://images.gr-assets.com/books/1360558233m...,the name of the wind the kingkiller chronicle 1,29.362416
236,32920226,328,"Sing, Unburied, Sing",4592,https://www.goodreads.com/book/show/32920226-s...,https://images.gr-assets.com/books/1499340866m...,sing unburied sing,23.428571
13156,36301023,17,"My Plain Jane (The Lady Janies, #2)",13,https://www.goodreads.com/book/show/36301023-m...,https://images.gr-assets.com/books/1507936746m...,my plain jane the lady janies 2,22.230769
13025,26857046,17,The Invisible Life of Addie La Rue,13,https://www.goodreads.com/book/show/26857046-t...,https://s.gr-assets.com/assets/nophoto/book/11...,the invisible life of addie la rue,22.230769
16306,35430702,14,The Lady's Guide to Petticoats and Piracy,9,https://www.goodreads.com/book/show/35430702-t...,https://s.gr-assets.com/assets/nophoto/book/11...,the ladys guide to petticoats and piracy,21.777778
216,30753987,342,The Leavers,5602,https://www.goodreads.com/book/show/30753987-t...,https://images.gr-assets.com/books/1489158974m...,the leavers,20.878972


In [31]:
all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
1,29983711,1089,Pachinko,8161,https://www.goodreads.com/book/show/29983711-p...,https://images.gr-assets.com/books/1462393298m...,pachinko,145.315648
238,4408,327,East of Eden,3447,https://www.goodreads.com/book/show/4408.East_...,https://images.gr-assets.com/books/1323882457m...,east of eden,31.020888
236,32920226,328,"Sing, Unburied, Sing",4592,https://www.goodreads.com/book/show/32920226-s...,https://images.gr-assets.com/books/1499340866m...,sing unburied sing,23.428571
216,30753987,342,The Leavers,5602,https://www.goodreads.com/book/show/30753987-t...,https://images.gr-assets.com/books/1489158974m...,the leavers,20.878972
249,8153988,322,"The Eye of the World (Wheel of Time, #1)",5740,https://www.goodreads.com/book/show/8153988-th...,https://images.gr-assets.com/books/1465920672m...,the eye of the world wheel of time 1,18.063415
441,33253215,236,The Heart's Invisible Furies,3629,https://www.goodreads.com/book/show/33253215-t...,https://images.gr-assets.com/books/1490803456m...,the hearts invisible furies,15.347479
71,30688435,533,Exit West,21378,https://www.goodreads.com/book/show/30688435-e...,https://images.gr-assets.com/books/1477324680m...,exit west,13.288848
251,32283423,321,American War,7776,https://www.goodreads.com/book/show/32283423-a...,https://images.gr-assets.com/books/1481494946m...,american war,13.251157
228,26025588,335,Behold the Dreamers,8793,https://www.goodreads.com/book/show/26025588-b...,https://images.gr-assets.com/books/1439643293m...,behold the dreamers,12.762993
345,17912498,266,The Queen of the Night,6115,https://www.goodreads.com/book/show/17912498-t...,https://images.gr-assets.com/books/1460425080m...,the queen of the night,11.570891


In [32]:
popular_recs = all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False)

In [33]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)


popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,book_count,title,ratings,url,cover_image,mod_title,score
236,32920226,328,"Sing, Unburied, Sing",4592,Goodreads,,sing unburied sing,23.428571
216,30753987,342,The Leavers,5602,Goodreads,,the leavers,20.878972
441,33253215,236,The Heart's Invisible Furies,3629,Goodreads,,the hearts invisible furies,15.347479
71,30688435,533,Exit West,21378,Goodreads,,exit west,13.288848
251,32283423,321,American War,7776,Goodreads,,american war,13.251157
228,26025588,335,Behold the Dreamers,8793,Goodreads,,behold the dreamers,12.762993
345,17912498,266,The Queen of the Night,6115,Goodreads,,the queen of the night,11.570891
325,28114515,276,The Wangs vs. the World,7044,Goodreads,,the wangs vs the world,10.814310
108,34273236,459,Little Fires Everywhere,21135,Goodreads,,little fires everywhere,9.968346
104,28815371,467,The Mothers,22346,Goodreads,,the mothers,9.759644
